## Introduction

The name of housing unit types are specific of the Italian market, therefore I create a short dictionary with its translation in English:
- Monolocale: studio flat(one-room apartment)
- Bilocale: two-rooms apartment       
- Trilocale: three-rooms apartment       
- Quadrilocale: four-rooms apartment
- Appartamento: flat/apartment (that is from four to more rooms) 
- Attico: attic           
- Villa: house/villa
- Palazzo: building/palace
- Mansarda: mansard        
- Loft: loft apartment       
- Terratetto: this is a typical italian building of the early year of the 20th century. It means "from the ground to the roof"      
- Open space: open space apartment  
- Casale: farmhouse

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import math
import time

# from sklearn.preprocessing import LabelEncoder
# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import GridSearchCV
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import  mean_absolute_error, r2_score, mean_absolute_percentage_error, mean_squared_error

# from xgboost import XGBRegressor

from lightgbm import LGBMRegressor

import warnings
warnings.filterwarnings('ignore')

## Data Acquisition

In [ ]:
df_italia = pd.read_csv('D001_RAW_DATASET.csv')

In [ ]:
df_italia.head()

In [ ]:
df_italia.shape

### Feature engineering

In [ ]:
# I create three new features City, Zone, and Housing unit from the original title description

df_italia['City'] = df_italia['Title description'].apply(lambda x: x.split()[-1])
df_italia['Zone'] = df_italia['Title description'].apply(lambda x: x.split(", ")[-2])

df_italia['Housing_unit'] = df_italia['Title description'].str.split(' ').str[0] 


In [ ]:
df_italia.head()

In [ ]:
df_italia['City'].nunique(), df_italia['Zone'].nunique(), df_italia['Housing_unit'].nunique()

## Data Wrangling and Data Cleaning

In [ ]:
df_italia.isnull().sum()

In [ ]:
df_italia = df_italia[df_italia.Price.isnull()==False] #removed rows with Nan Price
df_italia.drop(columns = 'N_toilettes', inplace = True) #dropped N_toilettes becuase it is missing in 7519 rows on 11395
df_italia = df_italia[df_italia.Surface.isnull()==False]

In [ ]:
df_italia.info()

In [ ]:
df_italia.N_rooms.unique()

In [ ]:
df_italia[df_italia.N_rooms.isnull()==True].groupby('Housing_unit')['City'].count()
# 'Albergo', 'Cascina', 'Soffitta', 'Terratetto'

In [ ]:
df_italia.loc[(df_italia['Housing_unit']=='Monolocale') & (df_italia['N_rooms'].isnull()==True), 'N_rooms'] = 1

In [ ]:
df_italia[df_italia.N_rooms.isnull()==True].groupby('Housing_unit')['City'].count()


In [ ]:
df_italia = df_italia[df_italia.N_rooms.isnull()==False]

In [ ]:
city_size = {
    "Roma": "big",
    "Milano": "big",
    "Napoli": "big",

    "Torino": "mid-big",
    "Palermo": "mid-big",
    "Genova": "mid-big",

    "Bologna": "mid",
    "Firenze": "mid",
    "Bari": "mid",
    "Catania": "mid",
    "Verona": "mid",
    "Venezia": "mid",
    "Messina": "mid",
    "Padova": "mid",
    "Trieste": "mid",
    "Brescia": "mid",
    "Taranto": "mid",
    "Prato": "mid",

    "Parma": "mid-small",
    "Modena": "mid-small"
}

df_italia["city_size"] = df_italia["City"].map(city_size)


#### Macroregion

In [ ]:
city_macroregion = {
    "Milano": "north",
    "Torino": "north",
    "Genova": "north",
    "Bologna": "north",
    "Verona": "north",
    "Venezia": "north",
    "Padova": "north",
    "Trieste": "north",
    "Parma": "north",
    "Brescia": "north",
    "Modena": "north",

    "Prato": "center",
    "Roma": "center",
    "Firenze": "center",

    "Napoli": "south",
    "Palermo": "south",
    "Bari": "south",
    "Catania": "south",
    "Messina": "south",
    "Taranto": "south"
}


df_italia["macroregion"] = df_italia["City"].map(city_macroregion)


#### Zone

In [ ]:
df_italia['Zone'].nunique() # it is a too granular data, I drop Zone to avoid overfitting in train

In [ ]:
df_italia.drop(columns =['Unnamed: 0','Index','Title description','Zone'], axis=1, inplace= True)

#### Housing_unit

In [ ]:
df_italia['Housing_unit'].value_counts()

In [ ]:
# Here, I fixed mistakes, aggregated into one when there were synonyms, 
# and deleted the element 'Albergo' (that is, Hotel) beacuse it is not a housing unit in which to live and so it is out of my research

df_italia["Housing_unit"] = df_italia["Housing_unit"].replace('Loft,', 'Loft')
df_italia["Housing_unit"] = df_italia["Housing_unit"].replace('Open', 'Open space')
df_italia["Housing_unit"] = df_italia["Housing_unit"].replace('Soffitta', 'Mansarda')
df_italia["Housing_unit"] = df_italia["Housing_unit"].replace(['Rustico', 'Cascina'], 'Casale')
df_italia["Housing_unit"] = df_italia["Housing_unit"].replace(['Casa'], 'Villa')
df_italia = df_italia[df_italia["Housing_unit"] != 'Albergo'] 


df_italia['Housing_unit'].value_counts()

#### Floor

In [ ]:
df_italia['Floor'].unique() # we can see that some floor numbers are interger, while others are strings

In [ ]:
#I calculated the mode in order to replace NaN values
df_italia["Floor"].mode()

In [ ]:
mapping = {'3': 3.0,
            '15': 15.0, 
            '8': 8.0, 
            '5': 5.0, 
            '1': 1.0, 
            '2': 2.0, 
            '4': 4.0, 
            'R': 0.5, 
            '0': 0.0, 
            '6': 6.0, 
            '7': 7.0, 
            '3 - 4': 4.0,
            '1 - 2' : 2.0, 
            '10': 10.0, 
            '4 - 5': 5.0, 
            '5 - 6': 6.0, 
            'nan': 1.0, 
            2 : 2.0, 
            6 : 6.0, 
            8 : 8.0, 
            0 : 0.0, 
            1 : 1.0, 
            3 : 3.0, 
            4 : 4.0, 
            5 : 5.0
            }


df_italia['floor'] = df_italia['Floor'].map(mapping)
df_italia.drop('Floor', axis = 1, inplace = True)


In [ ]:
df_floor = df_italia['floor'].value_counts().reset_index().sort_values(by = 'floor', ascending  = True)

plt.bar(df_floor['floor'], df_floor['count'])
plt.xlabel("floor")
plt.ylabel("Count");


#### N_rooms

In [ ]:
housing_to_rooms = {
    'Monolocale': '1',
    'Appartamento': '5',
    'Attico': '3',
    'Casale': '3',
    'Loft': '2',
    'Mansarda': '2',
    'Open space': '2',
    'Palazzo': '5+',
    'Terratetto': '5+',
    'Villa': '5+'
}

df_italia['N_rooms'] = df_italia['N_rooms'].fillna(
    df_italia['Housing_unit'].map(housing_to_rooms)
)


In [ ]:
mapping = {
                1: 1, 
                '2': 2, 
                '3': 3, 
                '4': 4, 
                '5': 5,
                '5+': 6
            }

df_italia['num_rooms'] = df_italia['N_rooms'].map(mapping)
df_italia.drop('N_rooms', axis = 1, inplace = True)

In [ ]:
df_italia

In [ ]:
df_rooms = df_italia['num_rooms'].value_counts().reset_index().sort_values(by = 'num_rooms', ascending  = True)

plt.bar(df_rooms['num_rooms'], df_rooms['num_rooms'])
plt.xlabel("num_rooms")
plt.ylabel("Count");


In [ ]:
df_italia.isnull().sum()

#### Price

In [ ]:
df_italia.Price.plot(kind = 'hist', bins = 100);

In [ ]:
# df_italia.Price.plot(kind = 'boxplot');
plt.boxplot(df_italia['Price']);


In [ ]:

bins = [-float("inf"), 450, 700, 1000, 2000, 4000, float("inf")]
labels = [
    "very low",
    "low",
    "mid-low",
    "mid-high",
    "high",
    "very high"
]

df_italia["rent_bracket"] = pd.cut(
    df_italia["Price"],
    bins=bins,
    labels=labels,
    right=False
)


In [ ]:
df_italia["rent_bracket"] = pd.Categorical(
    df_italia["rent_bracket"],
    categories=labels,
    ordered=True
)


In [ ]:
df_italia

#### Surface

In [ ]:
df_italia.Surface.plot(kind = 'hist', bins = 100);

In [ ]:
plt.boxplot(df_italia['Surface']);


In [ ]:
df_italia = df_italia[~(df_italia['Surface']<20)] # I erased those under 20 mq, bc they can only be error or not for humans

In [ ]:

bins = [-np.inf, 40, 70, 100, 200, 400, np.inf]
labels = [
    "very small",
    "small",
    "mid-small",
    "mid",
    "large",
    "very large"
]

df_italia["surface_bracket"] = pd.cut(
    df_italia["Surface"],
    bins=bins,
    labels=labels,
    right=False
)


In [ ]:
df_italia["surface_bracket"] = pd.Categorical(
    df_italia["surface_bracket"],
    categories=labels,
    ordered=True
)


# Fai una combo di queste quattro feature per irrubustire il pattern

In [ ]:
df_italia.groupby(['city_size', 'macroregion', 'rent_bracket', 'surface_bracket'])['num_rooms'].count().reset_index().sort_values(by="num_rooms", ascending =False)

## Explanatory Data Analysis

In [ ]:
df_italia.head()

#### Price

##### Observations:

- The average rent price is 1496€ per month, while the median value is 1100€.
- Most of the prices are below the 2,000€ per month, while a small percentange is above this threshold until picks between 8,000 and 10,000€ per month.
- The distribution of prices is left skewed.
- The most expensive housing units are avereagly: Villa, Palazzo, Attico, Terratetto and Appartamento; while the cheapest are Monolocale, Mansarda and Bilocale.
- Florence, Milan and Rome have the highest average rent price.
- The price was analyzed in relation with the five most frequent types of housing units: Monolocale, Bilocale, Trilocale, Quadrilocale, Appartmento.
- It emerged that the most costly are Appartamento and Quadrilocale, probably because they have the more rooms and a larger surface, followed by Trilocale. The cheapest are Monolocale and Bilocale.

In [ ]:
plt.figure(figsize=(10,5))
plt.title('Distribution of rent prices')
plt.hist(df_italia['Price'], bins = 100, color = 'firebrick')
plt.xlabel('Price in €')
plt.ylabel('Frequency');

In [ ]:
df_italia['Price'].mean(), df_italia['Price'].median()

In [ ]:
df_price = df_italia[['Housing_unit','Price']].groupby(['Housing_unit'])['Price'].mean().sort_values(ascending = False).round().reset_index()

plt.figure(figsize=(15, 8))
fig = sns.barplot(data = df_price, x = 'Housing_unit', y = 'Price')
plt.bar_label(fig.containers[0])
plt.title('Average price by type of housing unit');

In [ ]:
df_avg_price = df_italia[['City', 'Price']].groupby(['City'])['Price'].mean().sort_values(ascending = False).reset_index().round()

ax = plt.subplots(figsize=(23, 7))
splot = sns.barplot(data = df_avg_price, x = 'City', y = 'Price', ci = None)
plt.bar_label(splot.containers[0])
plt.title('Average rent price among Italian cities')
plt.ylabel('Average price in €');

In [ ]:
df_housing_unit_filtered = df_italia[((df_italia.Housing_unit == 'Monolocale') | (df_italia.Housing_unit == 'Bilocale') | (df_italia.Housing_unit == 'Trilocale') |
                      (df_italia.Housing_unit == 'Quadrilocale') | (df_italia.Housing_unit == 'Appartamento')) &
                      ((df_italia.City == 'Roma') | (df_italia.City == 'Milano') | (df_italia.City == 'Torino') |
                      (df_italia.City == 'Firenze') | (df_italia.City == 'Bologna') | 
                      (df_italia.City == 'Genova') | (df_italia.City == 'Napoli') | (df_italia.City == 'Palermo'))]

df_housing_unit_filtered = df_housing_unit_filtered.round()
df_housing_unit_filtered.head()

In [ ]:
fig, ax = plt.subplots(figsize=(23, 7))
labels_ordered = ['Monolocale','Bilocale','Trilocale', 'Quadrilocale','Appartamento']
cities_ordered = ['Milano','Roma', 'Torino', 'Firenze', 'Bologna', 'Genova', 'Napoli', 'Palermo']
splot = sns.barplot(data = df_housing_unit_filtered, x = 'City', y = 'Price', hue = 'Housing_unit', order = cities_ordered, hue_order = labels_ordered , ax = ax, ci=None)
ax.bar_label(splot.containers[0], rotation=300)
ax.bar_label(splot.containers[1], rotation=300)
ax.bar_label(splot.containers[2], rotation=300)
ax.bar_label(splot.containers[3], rotation=300)
ax.bar_label(splot.containers[4], rotation=300)
plt.margins(y = 0.125)
plt.title('Average price in € by type of housing unit in the 8 major Italian cities')
plt.ylabel('Average price in €');

#### Surface

##### Observations:

- The average surface is 97 m², while the median value is 78 m².
- The cities with the highest average surface are Firenza, Catania and Roma, while those with the lowest are Torino, Bologna, and Trieste
- The housing unit type with the largest average surface are Villa, Terratetto and Appartamento, while those with the lowest are Monolocale, Bilocale and Mansarda.

In [ ]:
plt.figure(figsize=(10,5))
plt.title('Distribution of rent surface')
plt.hist(df_italia['Surface'], bins = 100, color = 'firebrick')
plt.xlabel('Surface in m²')
plt.ylabel('Frequency');

In [ ]:
round(df_italia['Surface'].mean(),0), df_italia['Surface'].median()

In [ ]:
plt.figure(figsize=(10,5))
plt.scatter(df_italia['Surface'],df_italia['Price'], color = 'firebrick')
plt.xlabel('Surface in m²')
plt.ylabel('Price in €');

In [ ]:
df_avg_surface_city = df_italia[['City','Surface']].groupby(['City'])['Surface'].mean().sort_values(ascending=False).reset_index()

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(data = df_avg_surface_city, x = 'City', y = 'Surface')
plt.xticks(rotation = 60)
plt.title('Top 20 largest italian cities ordered by average surface of rents housing unit');

In [ ]:
df_avg_surface_housing_unit = round(df_italia[['City', 'Housing_unit', 'Surface']].groupby(['Housing_unit'])['Surface'].mean().sort_values(ascending=False).reset_index(),0)
df_avg_surface_housing_unit

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(data = df_avg_surface_housing_unit, x = 'Housing_unit', y = 'Surface')
plt.xticks(rotation = 60)
plt.title('Housing Unit types ordered by average surface');

In [ ]:
fig, ax = plt.subplots(figsize=(23, 7))
labels_ordered = ['Monolocale','Bilocale','Trilocale', 'Quadrilocale','Appartamento']
cities_ordered = ['Milano','Roma', 'Torino', 'Firenze', 'Bologna', 'Genova', 'Napoli', 'Palermo']
splot = sns.barplot(data = df_housing_unit_filtered, x = 'City', y = 'Surface', hue = 'Housing_unit', order = cities_ordered, hue_order = labels_ordered , ax = ax, ci=None)

plt.margins(y = 0.125)
plt.title('Average surface in m² by type of housing unit in the 8 major Italian cities')
plt.ylabel('Average surface in m²');

#### N_rooms

##### Observations:

- the most frequent number of rooms in a renting house: 2-rooms (3676), 3-rooms (2902), 4-rooms (1671) 

- Cities with most:
        
        - Monolocale-type flat: Torino(228), Milano(183), Bologna (161)
        
        - Bilocale-type flat: Milano (897), Torino (551), Roma (541)
        
        - Trilocale-type flat: Milano (515), Roma (512), Torino (352)
        
        - Quadrilocale-type flat: Roma (261), Milano (172), Torino (162)
        
        - Apartments: Roma (269), Firenze (263), Torino(164), Genova (163)

In [ ]:
rooms = df_italia.groupby(['num_rooms']).size().reset_index().rename(columns = {0: 'Num'})
rooms

In [ ]:
plt.figure(figsize=(10,5))
plt.title('Frequency of number of rooms by rent')
fig = plt.bar(rooms['num_rooms'], rooms['Num'], color = 'firebrick')

plt.xlabel('Number of rooms')
plt.ylabel('Frequency');

In [ ]:
# Here, I found the most frequent number of rooms by city
df_italia[['City','num_rooms']].groupby(['City'])['num_rooms'].agg(pd.Series.mode)

In [ ]:
df_monolocali = df_italia[df_italia['Housing_unit']=='Monolocale']
df_monolocali[['City','num_rooms']].groupby(['City'])['num_rooms'].count().sort_values(ascending = False).reset_index().head(5)

In [ ]:
df_bilocali = df_italia[df_italia['Housing_unit']=='Bilocale']
df_bilocali[['City','num_rooms']].groupby(['City'])['num_rooms'].count().sort_values(ascending = False).reset_index().head(5)

In [ ]:
df_trilocali = df_italia[df_italia['Housing_unit']=='Trilocale']
df_trilocali[['City','num_rooms']].groupby(['City'])['num_rooms'].count().sort_values(ascending = False).reset_index().head(5)

In [ ]:
df_trilocali = df_italia[df_italia['Housing_unit']=='Trilocale']
df_trilocali[['City','num_rooms']].groupby(['City'])['num_rooms'].count().sort_values(ascending = False).reset_index().head(5)

In [ ]:
df_locals = df_italia[['City','Housing_unit','num_rooms']].groupby(['Housing_unit','City'])['num_rooms'].count().reset_index()

df_locals = df_locals[((df_locals.Housing_unit == 'Monolocale') | (df_locals.Housing_unit == 'Bilocale') | (df_locals.Housing_unit == 'Trilocale') |
                      (df_locals.Housing_unit == 'Quadrilocale') | (df_locals.Housing_unit == 'Appartamento')) &
                      ((df_locals.City == 'Roma') | (df_locals.City == 'Milano') | (df_locals.City == 'Torino') |
                      (df_locals.City == 'Firenze') | (df_locals.City == 'Bologna') | 
                      (df_locals.City == 'Genova') | (df_locals.City == 'Napoli') | (df_locals.City == 'Palermo'))]
df_locals.head(5)

In [ ]:
fig, ax = plt.subplots(figsize=(23, 7))
labels_ordered = ['Monolocale','Bilocale','Trilocale', 'Quadrilocale','Appartamento']
cities_ordered = ['Milano','Roma', 'Torino', 'Firenze', 'Bologna', 'Genova', 'Napoli', 'Palermo']
splot = sns.barplot(data = df_locals, x = 'City', y = 'num_rooms', hue = 'Housing_unit', order = cities_ordered, hue_order = labels_ordered , ax = ax)
plt.bar_label(splot.containers[0])
plt.bar_label(splot.containers[1])
plt.bar_label(splot.containers[2])
plt.bar_label(splot.containers[3])
plt.bar_label(splot.containers[4])
plt.title('Type of housing unit in the 8 major Italian cities')
plt.ylabel('Number of hounsing units');

#### City

##### Observations:

- The cities with most announcements on Immobiliare.it are Roma, Milano, and Torino; while those the less announcements are Prato, Taranto, and Bari
- The cities with the most expansive average rent price are Firenze, Milano, and Roma; while those with the lowest are Taranto, Genova, Messina

In [ ]:
city = df_italia.groupby(['City']).size().sort_values(ascending = False).reset_index().rename(columns = {0: 'Num'})
city

In [ ]:
plt.figure(figsize=(10,5))
plt.title('Numerosity of rents by city')
fig = plt.bar(city['City'], city['Num'], color = 'firebrick')
plt.xlabel('Cities')
plt.ylabel('Number of rents by city')
plt.xticks(rotation = 45);

In [ ]:
df_most_expensive_cities = df_italia[['City','Price']].groupby(['City'])['Price'].mean().round().reset_index().sort_values(by ='Price', ascending=False)

plt.figure(figsize=(15,5))
fig = sns.barplot(data = df_most_expensive_cities, x='City', y='Price', color='firebrick')
plt.bar_label(fig.containers[0])
plt.title('Cities with the most expensive average rent price')
plt.ylabel('Price in €')
plt.xticks(rotation = 60);

#### Housing_unit

Here, there are some graphs that are specific for the variable Housing_unit but that have been already analyzed above

In [ ]:
N_types_of_locals = df_italia.groupby(['Housing_unit']).size().sort_values(ascending = False).reset_index().rename(columns={0:'Num'})
N_types_of_locals

In [ ]:
plt.figure(figsize=(10,5))
plt.title('Numerosity of rents by type of locals')
plt.bar(N_types_of_locals['Housing_unit'], N_types_of_locals['Num'], color = 'firebrick')
plt.xlabel('Types of housing unit')
plt.ylabel('Numerosity')
plt.xticks(rotation = 45);

In [ ]:
df_italia1 = df_italia[['City', 'Housing_unit','num_rooms']].groupby(['City','Housing_unit'])['num_rooms'].count().sort_values(ascending = False).reset_index()

df1 = df_italia1[(df_italia1['Housing_unit']=='Appartamento') | (df_italia1['Housing_unit']=='Monolocale') | (df_italia1['Housing_unit']=='Bilocale') |
                (df_italia1['Housing_unit']=='Trilocale') | (df_italia1['Housing_unit']=='Quadrilocale')]

In [ ]:
plt.figure(figsize=(23,5))
labels = ['Monolocale', 'Bilocale', 'Trilocale', 'Quadrilocale','Appartamento']
plt.legend(labels =['Monolocale', 'Bilocale', 'Trilocale', 'Quadrilocale','Appartamento'], loc=1)
sns.barplot(data = df1, x = 'City', y = 'num_rooms', hue = 'Housing_unit', hue_order = labels)
plt.title('Most common housing units among Italian cities')
plt.ylabel('Num of units');

In [ ]:
df_price = df_italia[['Housing_unit','Price']].groupby(['Housing_unit'])['Price'].mean().sort_values(ascending = False).round().reset_index()

plt.figure(figsize=(15, 8))
fig = sns.barplot(data = df_price, x = 'Housing_unit', y = 'Price')
plt.bar_label(fig.containers[0])
plt.ylabel('Price in €')
plt.title('Average price in € by type of housing unit');

In [ ]:
df_monolocali = df_italia[df_italia['Housing_unit']=='Monolocale']
df_monolocali[['City','num_rooms']].groupby(['City'])['num_rooms'].size().sort_values(ascending = False).reset_index().head(5)

In [ ]:
df_bilocali = df_italia[df_italia['Housing_unit']=='Bilocale']
df_bilocali[['City','num_rooms']].groupby(['City'])['num_rooms'].size().sort_values(ascending = False).reset_index().head(5)

In [ ]:
df_trilocali = df_italia[df_italia['Housing_unit']=='Trilocale']
df_trilocali[['City','num_rooms']].groupby(['City'])['num_rooms'].size().sort_values(ascending = False).reset_index().head(5)

In [ ]:
df_quadrilocali = df_italia[df_italia['Housing_unit']=='Quadrilocale']
df_quadrilocali[['City','num_rooms']].groupby(['City'])['num_rooms'].size().sort_values(ascending = False).reset_index().head(5)

## CREATE FINAL DATASET

In [ ]:
df_italia

In [ ]:
df_italia.to_csv("D002_CLEANED_DATASET_RENT_ITALY.csv", index= False)